<a href="https://colab.research.google.com/github/guptamols/zero_to_hero_nn/blob/main/micro_grad_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive",force_remount=True)

Mounted at /content/gdrive


In [2]:
import os
os.chdir('gdrive/MyDrive/zero_to_hero_nn/zero_to_hero_nn')

In [15]:
! git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	micro_grad_practice.ipynb

nothing added to commit but untracked files present (use "git add" to track)


### References
https://github.com/AayushSameerShah/Neural-Net-Zero-to-Hero-with-Andrej

https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b

https://medium.com/@0ssamaak0/how-open-ais-andrej-karpathy-made-one-of-the-best-tutorials-in-deep-learning-e6b6445a2d05

https://github.com/karpathy/nn-zero-to-hero/tree/master

https://colab.research.google.com/drive/11SbzRCfRAXyveiri8CJ7Bjbn4MfiyvsL#scrollTo=yk3QDMPjWyYp

https://github.com/Anri-Lombard/micrograd

In [3]:
! pwd

/content/gdrive/MyDrive/zero_to_hero_nn/zero_to_hero_nn


In [13]:
! git remote set-url origin git@github.com:'guptamols'/'https://github.com/guptamols/zero_to_hero_nn.git'

In [19]:
! git remote -v

origin	https://github.com/guptamols/zero_to_hero_nn.git (fetch)
origin	https://github.com/guptamols/zero_to_hero_nn.git (push)


In [18]:
! git remote set-url origin https://github.com/guptamols/zero_to_hero_nn.git

In [22]:
! git push origin main

Everything up-to-date
